In [15]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import subplot
from tqdm import tqdm
import h5py 
import os
import re
import subprocess
import pandas as pd

In [16]:
folder_path = "../"
write_path = "../gwtc3_converted/"

PLANCK_2018_Ho = 2.1816926176539463e-18 ### CGS
PLANCK_2018_OmegaMatter = 0.3158
PLANCK_2018_OmegaLambda = 1. - PLANCK_2018_OmegaMatter
PLANCK_2018_OmegaRadiation = 0.

In [17]:
# Read the data
files = (file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file)))
events_files = list(filter(lambda x: x.endswith((".h5", ".hdf5")), files))
for i in range(len(events_files)):print(f"{i+1}. {events_files[i]} (i = {i})")

1. IGWN-GWTC3p0-v1-GW200225_060421_PEDataRelease_mixed_cosmo.h5 (i = 0)
2. IGWN-GWTC2p1-v2-GW190521_074359_PEDataRelease_mixed_cosmo.h5 (i = 1)
3. IGWN-GWTC2p1-v2-GW190915_235702_PEDataRelease_mixed_cosmo.h5 (i = 2)
4. IGWN-GWTC3p0-v1-GW200224_222234_PEDataRelease_mixed_cosmo.h5 (i = 3)
5. IGWN-GWTC2p1-v2-GW190706_222641_PEDataRelease_mixed_cosmo.h5 (i = 4)
6. IGWN-GWTC2p1-v2-GW190412_053044_PEDataRelease_mixed_cosmo.h5 (i = 5)
7. IGWN-GWTC2p1-v2-GW190929_012149_PEDataRelease_mixed_cosmo.h5 (i = 6)
8. IGWN-GWTC3p0-v1-GW200209_085452_PEDataRelease_mixed_cosmo.h5 (i = 7)
9. IGWN-GWTC3p0-v1-GW200219_094415_PEDataRelease_mixed_cosmo.h5 (i = 8)
10. IGWN-GWTC2p1-v2-GW190828_065509_PEDataRelease_mixed_cosmo.h5 (i = 9)
11. IGWN-GWTC3p0-v1-GW191129_134029_PEDataRelease_mixed_cosmo.h5 (i = 10)
12. IGWN-GWTC3p0-v1-GW191109_010717_PEDataRelease_mixed_cosmo.h5 (i = 11)
13. IGWN-GWTC2p1-v2-GW190408_181802_PEDataRelease_mixed_cosmo.h5 (i = 12)
14. IGWN-GWTC2p1-v2-GW190707_093326_PEDataRelease_mixed_c

In [18]:
if not os.path.exists(write_path + "event-list.txt"):
    with open(write_path + "event-list.txt", "w") as f:
        pass

In [19]:
#regex
re1 = r"GWTC-1"
re2= r"GWTC2p1"
re3 = r"GWTC3"

In [20]:
event_waveforms = []
compile_df = pd.DataFrame(columns=["event", "waveform", "cosmology", "prior_cosmology1", "prior_cosmology2", "prior_a_1 (or chi1)", "prior_a_2 (or chi2)", "prior_tilt_1", "prior_tilt_2"])


for i in tqdm(range(len(events_files))): 
    file_path = events_files[i]
    event_file = folder_path+file_path
    with h5py.File(event_file,'r+') as file: 
        wave_index = 0
        if i in [14]:
            wave_index = 2
        waveform = list(file.keys())[wave_index]
        # print(file_path)
        # print(file.keys())
        # print(file[waveform].keys())
        # print(file[waveform]["priors"].keys())
        # print(file[waveform]["meta_data"]['meta_data'].keys())
        # print(file[waveform]["description"][:])
        # print(file[waveform]["priors"]["analytic"].keys())
        # print(file[waveform]["meta_data"]['meta_data'].keys())
        # print(file[waveform]["priors"]['analytic']["luminosity_distance"][:][0])
        # print(file[waveform]["priors"]['analytic']["a_1"][:][0])
        # print(compile_df.columns)
        # print(list_row)
        # print(file[waveform]["posterior_samples"].dtype.descr)

        if i in [13, 23, 25, 41, 46, 61]:
            spin1_used = "NOT INCLUDED (Uniform)" ; spin2_used = "NOT INCLUDED (Uniform)"      
            tilt1_used = "NOT INCLUDED (Isotropic)" ; tilt2_used = "NOT INCLUDED (Isotropic)"
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            cosmo_prior = "NOT INCLUDED"
        elif i == 42: # GW190425 - https://arxiv.org/pdf/2108.01045
            spin1_used = "chi < 0.89"; spin2_used = "chi < 0.89"   
            tilt1_used = "NOT INCLUDED (Isotropic)" ; tilt2_used = "NOT INCLUDED (Isotropic)"
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            cosmo_prior = "NOT INCLUDED"
        elif i == 48: # GW170817 - https://arxiv.org/pdf/1805.11579
            # print(file["IMRPhenomPv2NRT_highSpin_prior"]["costilt1"])
            # print(file[waveform].dtype.descr)
            cosmology_used = "planck 2015"
            cosmo_prior = "NOT INCLUDED"
            spin1_used = "chi < 0.89"; spin2_used = "chi < 0.89"
            tilt1_used = "NOT INCLUDED (Isotropic)"; tilt2_used = "NOT INCLUDED (Isotropic)"
        else:
            cosmo_prior = file[waveform]["priors"]['analytic']["luminosity_distance"][:][0]
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            spin1_used = str(file[waveform]["priors"]['analytic']["a_1"][:][0]) ; spin2_used = str(file[waveform]["priors"]['analytic']["a_2"][:][0])
            tilt1_used = str(file[waveform]["priors"]['analytic']["tilt_1"][:][0]) ; tilt2_used = str(file[waveform]["priors"]['analytic']["tilt_2"][:][0])
            
        list_row = [events_files[i], waveform, cosmology_used, cosmo_prior, spin1_used, spin2_used, tilt1_used, tilt2_used]
        event_waveforms.append(f"Converting {waveform} from {event_file}")
                                               
            
    args = [f"{waveform}", f"{event_file}",
            "--Ho", f"{PLANCK_2018_Ho}", "--OmegaMatter", f"{PLANCK_2018_OmegaMatter}", "--OmegaRadiation",
            f"{PLANCK_2018_OmegaRadiation}", "--OmegaLambda", f"{PLANCK_2018_OmegaLambda}", "-o", f"{write_path}"]
    
    if re.search(re1, file_path):
        cosmo_cat = "d_l^2"  # According to Amanda's O4a work
        subprocess.run(["python", "h52csv_GWTC1.py", *args])
    elif re.search(re2, file_path):
        cosmo_cat = "uniform in comoving volume"
        subprocess.run(["python", "h52csv_GWTC2p1.py", *args])
    elif re.search(re3, file_path):
        cosmo_cat = "uniform in comoving volume"
        subprocess.run(["python", "h52csv_GWTC3.py", *args])
    else:
        raise ValueError("No match found")
    
    list_row = [events_files[i], waveform, cosmology_used, cosmo_cat, cosmo_prior, spin1_used, spin2_used, tilt1_used, tilt2_used]
    compile_df.loc[i] = list_row
    
compile_df.to_csv("compile_df.csv", index=False)
compile_df

100%|██████████| 66/66 [02:52<00:00,  2.61s/it]


,event,waveform,cosmology,prior_cosmology1,prior_cosmology2,prior_a_1 (or chi1),prior_a_2 (or chi2),prior_tilt_1,prior_tilt_2
0,IGWN-GWTC3p0-v1-GW200225_060421_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(minimum=0, maximum=3.141592653589793, n...","b""Sine(minimum=0, maximum=3.141592653589793, n..."
1,IGWN-GWTC2p1-v2-GW190521_074359_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=10, maximum=10000,...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
2,IGWN-GWTC2p1-v2-GW190915_235702_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=10, maximum=10000,...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(minimum=0, maximum=3.141592653589793, n...","b""Sine(minimum=0, maximum=3.141592653589793, n..."
3,IGWN-GWTC3p0-v1-GW200224_222234_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(minimum=0, maximum=3.141592653589793, n...","b""Sine(minimum=0, maximum=3.141592653589793, n..."
4,IGWN-GWTC2p1-v2-GW190706_222641_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
...,...,...,...,...,...,...,...,...,...
61,IGWN-GWTC2p1-v2-GW190720_000836_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,NOT INCLUDED,NOT INCLUDED (Uniform),NOT INCLUDED (Uniform),NOT INCLUDED (Isotropic),NOT INCLUDED (Isotropic)
62,IGWN-GWTC2p1-v2-GW170729_185629_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
63,IGWN-GWTC3p0-v1-GW200112_155838_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
64,IGWN-GWTC2p1-v2-GW190910_112807_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."


In [21]:
     #  i = 52
# file_path = events_files[i]
# event_file = folder_path+file_path
# 
# with h5py.File(event_file,'r+') as file: 
#         waveform = list(file.keys())[0]
#         event_waveforms.append(f"Converting {waveform} from {event_file}")
#         
# args = [f"{waveform}", f"{event_file}",
#         "--Ho", f"{PLANCK_2018_Ho}", "--OmegaMatter", f"{PLANCK_2018_OmegaMatter}", "--OmegaRadiation",
#         f"{PLANCK_2018_OmegaRadiation}", "--OmegaLambda", f"{PLANCK_2018_OmegaLambda}", "-o", f"{write_path}", "-v"]
# 
# if re.search(re1, file_path):
#     subprocess.run(["python", "h52csv_GWTC1.py", *args])
# elif re.search(re2, file_path):
#     subprocess.run(["python", "h52csv_GWTC2p1.py", *args])
# elif re.search(re3, file_path):
#     subprocess.run(["python", "h52csv_GWTC3.py", *args])
# else:
#     raise ValueError("No match found")

In [22]:
# read txt file
event_txt_files = []
with open(write_path + "event-list.txt", "r") as f:
    event_txt_files = f.readlines()

print("Number of events converted: ", len(event_txt_files))
print("Number of events with errors: ", len(file_open_error)+len(file_convert_error))
print("Total number of events: ", len(events_files))

Number of events converted:  66


NameError: name 'file_open_error' is not defined

In [ ]:
# Read events and confirm the conversion. 
import numpy as np
event_txt_files = []
with open(write_path + "event-list.txt", "r") as f:
    event_txt_files = f.read().splitlines()
event_txt_files

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

for i in tqdm(range(len(event_txt_files))):
    try:
        event = event_txt_files[i]
        df = pd.DataFrame(np.genfromtxt(write_path + event, delimiter=",", names=True))
        fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 4))
        ax[0].hist(df["mass1_source"]); ax[0].set_xlabel("mass1_source")
        ax[1].hist(df["mass2_source"]); ax[1].set_xlabel("mass2_source")
        ax[2].hist(df["luminosity_distance"]); ax[2].set_xlabel("luminosity_distance")
        plt.suptitle(event[:8] + f" - PE Size: {df.shape[0]}")
        plt.savefig("verify_PE/" + event[:8] + ".png")
        plt.close()
    except:
        print(f"Problem at i = {i} the following event", event)
        continue

In [ ]:
i = 52
event = event_txt_files[i]
print(event)
df = pd.DataFrame(np.genfromtxt(write_path + event_txt_files[i], delimiter=",", names=True))
df_normal = pd.DataFrame(np.genfromtxt(write_path + event_txt_files[50], delimiter=",", names=True))
print(df.columns)
print(df_normal.columns) 
#Additionally includes geocenter_time, lnprob_geocenter_time, lnprob_spin1x_spin1y_spin1z, lnprob_spin2x_spin2y_spin2z, loglikelihood, luminosity_distance, spin1x, spin1y, spin1z, spin2x, spin2y, spin2z. (This has changed now but was true before modification of h52csv_GWTC1.py)

In [ ]:
# DEBUGGING: Manually read the PE file for GW170817 and check to see if there are spin and Dl components, change the reading conversion script for GWTC1 to include these components
i = 48
file_path = events_files[i]
event_file = folder_path+file_path
print(file_path)

with h5py.File(event_file, "r") as f:
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]
    # print(type(f[a_group_key])) 
    data = f[a_group_key]
    print(data.dtype.descr)

with h5py.File(event_file,'r+') as file: 
        waveform = list(file.keys())[0]
        event_waveforms.append(f"Converting {waveform} from {event_file}")
        
args = [f"{waveform}", f"{event_file}",
        "--Ho", f"{PLANCK_2018_Ho}", "--OmegaMatter", f"{PLANCK_2018_OmegaMatter}", "--OmegaRadiation",
        f"{PLANCK_2018_OmegaRadiation}", "--OmegaLambda", f"{PLANCK_2018_OmegaLambda}", "-o", f"{write_path}"]

subprocess.run(["python", "h52csv_GWTC1.py", *args])


df = pd.DataFrame(np.genfromtxt(write_path + event_txt_files[i], delimiter=",", names=True))
# df.columns

# Conversion of Injections

In [ ]:
vt_folder = "../../vt/"
vt_file = "o1+o2+o3_mixture_real+semianalytic-LIGO-T2100377-v2.hdf5"
vt_file_O3_full = "endo3_mixture-LIGO-T2100113-v12.hdf5"
vt_path = vt_folder + vt_file

In [ ]:
with h5py.File(vt_folder + vt_file_O3_full, "r") as f:
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]
    print(type(f[a_group_key])) 
    data = list(f[a_group_key])
    print(data)
    
with h5py.File(vt_path, "r") as f:
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]
    print(type(f[a_group_key])) 
    data = list(f[a_group_key])
    print(data)

In [ ]:
with h5py.File(vt_path,'r+') as file: 
        waveform = list(file.keys())[0]
        event_waveforms.append(f"Converting {waveform} from {vt_path}")
        
args = ["--max-far", f"{1}",
        "--Ho", f"{PLANCK_2018_Ho}", 
        "--OmegaMatter", f"{PLANCK_2018_OmegaMatter}", 
        "--OmegaRadiation", f"{PLANCK_2018_OmegaRadiation}", 
        "--OmegaLambda", f"{PLANCK_2018_OmegaLambda}", 
        f"{vt_path}",
        # "-v", 
        # f"{vt_folder}" + "sensitivity-estimate_O3.csv.gz"]
        f"{vt_folder}" + "sensitivity-estimate.csv.gz"]

subprocess.run(["python", "h52csv_injections.py", *args])
print("Done")

In [ ]:
path = f"{vt_folder}" + "sensitivity-estimate.csv.gz"
df = pd.read_csv(path)
df # Checking injections